<a href="https://colab.research.google.com/github/4060E009/AI_and_security/blob/master/10_3%E5%88%86%E5%89%B2%E6%87%89%E7%94%A8%E5%90%88%E4%BD%B52020_11_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://nbviewer.jupyter.org/github/LearnXu/pydata-notebook/blob/master/Chapter-10/10.3%20Apply%EF%BC%9AGeneral%20split-apply-combine%EF%BC%88%E5%BA%94%E7%94%A8%EF%BC%9A%E9%80%9A%E5%B8%B8%E7%9A%84%E5%88%86%E5%89%B2-%E5%BA%94%E7%94%A8-%E5%90%88%E5%B9%B6%EF%BC%89.ipynb

# **General split-apply-combine**

**general-purpose: 可以理解為通用，泛用。**

例子：在計算機軟件中，通用編程語言(General-purpose programming language )**指被設計為各種應用領域服務的編程語言。通常通用編程語言不含有為特定應用領域設計的結構。**

**相對而言，特定域編程語言就是為某一個特定的領域或應用軟件設計的編程語言。比如說，LaTeX就是專門為排版文獻而設計的語言。**

最通用的GroupBy(分組)**方法是apply，這也是本節的主題。如下圖所示，apply會把對象分為多個部分，然後將函數應用到每一個部分上，然後把所有的部分都合並起來**



**返回之前提到的tipping數據集，假設我們想要根據不同組（group），選擇前5個tip_pct值最大的。首先，寫一個函數，函數的功能為在特定的列，選出有最大值的行**

In [1]:
import numpy as np
import pandas as pd

In [2]:
!wget https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv

--2020-11-04 03:38:41--  https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6545 (6.4K) [text/plain]
Saving to: ‘tips.csv’

tips.csv            100%[===================>]   6.39K  --.-KB/s    in 0s      

2020-11-04 03:38:41 (67.5 MB/s) - ‘tips.csv’ saved [6545/6545]



In [5]:
tips = pd.read_csv('tips.csv')

In [6]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [7]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [8]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [9]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


**現在，如果我們按smoker分組，然後用apply來使用這個函數，我們能得到下面的結果**

In [10]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

**如果傳遞一個函數給apply，可以在函數之後，設定其他一些參數**

In [11]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

**試一試在GroupBy對象上調用describe**

In [12]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [13]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [14]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

**在GroupBy內部，當我們想要調用一個像describe這樣的函數的時候，其實相當於下面的寫法：**

**f = lambda x: x.describe()**

**grouped.apply(f)**


# **Suppressing the Group Keys**

**我們可以在groupby函數中設置group_keys=False來關閉這個功能**

In [15]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


# **Quantile and Bucket Analysis**

**把函數通過groupby整合起來，可以在做桶分析或分位數分析的時候更方便。假設一個簡單的隨機數據集和一個等長的桶類型（bucket categorization），使用cut**

In [16]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                       'data2': np.random.randn(1000)})
frame.head()

,data1,data2
0,2.372897,-1.703250
1,1.159432,0.926097
2,0.768928,-1.519208
3,1.214234,-0.126483
4,-1.030624,-1.101032


In [17]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0      (1.371, 2.889]
1     (-0.148, 1.371]
2     (-0.148, 1.371]
3     (-0.148, 1.371]
4    (-1.666, -0.148]
5      (1.371, 2.889]
6     (-0.148, 1.371]
7    (-1.666, -0.148]
8     (-0.148, 1.371]
9     (-3.19, -1.666]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.19, -1.666] < (-1.666, -0.148] < (-0.148, 1.371] <
                                    (1.371, 2.889]]

**cut返回的Categorical object（類別對象）能直接傳入groupby。所以我們可以在data2列上計算很多統計值**

In [18]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}


In [19]:
grouped = frame.data2.groupby(quartiles)

In [20]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.19, -1.666]",-1.717200,2.144720,40.0,-0.193085
"(-1.666, -0.148]",-2.866082,2.958925,388.0,-0.031046
"(-0.148, 1.371]",-2.572689,2.672097,494.0,-0.042573
"(1.371, 2.889]",-2.410051,1.801265,78.0,-0.008560


**也有相同長度的桶（equal-length buckets）；想要按照樣本的分位數得到相同長度的桶，用qcut。這里設定labels=False來得到分位數的數量**

In [21]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)